# Building a Database for Crime Reports Using PostgreSQL (psycopg2)

In this project, we'll be building a database for storing data related with crimes that occurred in Boston. This dataset is available in the file `boston.csv`.

The goal of this project is to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `boston.csv` file. We'll be creating the table inside a schema named `crimes` and will also create two user groups:

* `readonly`: permission to read data only
* `readwrite` permission to read and alter data only

Finally, we'll also need to create one user for each of these groups.

In [1]:
# Import libraries we'll use
import psycopg2
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns

# Show graphs in notebook
%matplotlib inline

# Graph formatting
sns.set(style='white', context='talk')
plt.style.use('dark_background')

## 1. Creating the database and schema

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

In [2]:
# Create the database, crime_db
conn = psycopg2.connect('dbname=dq user=dq')
cur = conn.cursor()
conn.autocommit = True

cur.execute('CREATE DATABASE crime_db;')
conn.autocommit = False
conn.close()

ProgrammingError: database "crime_db" already exists


In [3]:
# Connect to the new database, crime_db, and create schema called crimes.
conn = psycopg2.connect('dbname=crime_db user=dq')
cur = conn.cursor()

cur.execute("CREATE SCHEMA crimes;")

## 2. Exploring the dataset
We now have a database and a schema — we are ready to start creating tables.

Before we do that, we'll gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

In [4]:
# Read in the data and explore the first few rows
boston = pd.read_csv('boston.csv')
boston.info()
boston.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298329 entries, 0 to 298328
Data columns (total 7 columns):
incident_number    298329 non-null int64
offense_code       298329 non-null int64
description        298329 non-null object
date               298329 non-null object
day_of_the_week    298329 non-null object
lat                298329 non-null float64
long               298329 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 15.9+ MB


,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300
2,3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.346589,-71.072429
3,4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.334182,-71.078664
4,5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.275365,-71.090361


In [5]:
# Basic statistics of the numerical columns
boston.describe()

,incident_number,offense_code,lat,long
count,298329.000000,298329.000000,298329.000000,298329.000000
mean,149165.000000,2296.293143,42.322298,-71.082850
std,86120.308566,1183.110778,0.031881,0.029771
min,1.000000,111.000000,42.232413,-71.178674
25%,74583.000000,802.000000,42.297555,-71.097193
50%,149165.000000,2907.000000,42.325610,-71.077562
75%,223747.000000,3201.000000,42.348624,-71.062563
max,298329.000000,3831.000000,42.395042,-70.963676


Given their max valyes, we can set the `incident_number` column to the `INTEGER` datatype and the `offense_code` column to the `INT2` datatype. `SMALLINT`/`INT2` is a 2 byte integer that appropriate when only requiring a small range of integers.

We'll set the `lat` and `lon` columns to the `DECIMAL` datatype.

In [6]:
# Save the header, first row, and dataset as lists
with open('boston.csv') as file:
    reader = csv.reader(file)
    rows = list(reader)
    col_headers = rows[0]
    first_row = rows[1]
    crime_data = rows[1:]
    
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


### 2.1 Distinct values in columns

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. We begin to do this by identifiying the number of unique values in each column. This will be useful for checking whether an enumerated datatype might be a good choice for representing a column.

In [7]:
# Calculate the number of unique calues for each column
values_per_col = {}
for col in boston.columns:
    values_per_col[col] = len(boston[col].unique())
values_per_col

{'date': 1177,
 'day_of_the_week': 7,
 'description': 239,
 'incident_number': 298329,
 'lat': 18177,
 'long': 18177,
 'offense_code': 219}

The `day_of_the_week` column is a good candidate to use an enumerated datatype since there are only 7 possible values and they won't be changing any time soon. 

The `description` and `offense_code` columns could also be set as an enumerated datatype since they each contain just over 200 unique values and there are a limited number of laws to break. However, we'll stick to `VARCHAR` and `INTEGER` datatypes for now.

### 2.2 Max length of values in text columns

Computing the maximum length of any text-like column is useful for selecting the appropriate sizes for `VARCHAR` columns.

There are two textual column in the data set, namely, the `description` and `day_of_the_week` columns. However the day of the week contains only 7 values, one for each day. We can tell that the longest of them is "Wednesday" without needing any computation.

Let's compute the maximum length of each value in the `description` column.

In [8]:
# Calculate the max value length for the description text column
description_lengths = boston['description'].apply(lambda x: len(x))
print(max(description_lengths))

58


The longest value in the `description` column is 58 characters in length. We'll limit the size of the description to 100 characters to be on the safe side and use the `VARCHAR(100)` datatype to this column. 

## 3. Creating enumerated datatype

We'll create an enumerated datatype for the `day_of_the_Week` column since there are only 7 possible values.

In [9]:
# Turn list of weekdays into a string for easy insertion into query
weekdays = "'" +  "', '".join(boston['day_of_the_week'].unique()) + "'"
weekdays

"'Sunday', 'Tuesday', 'Monday', 'Saturday', 'Friday', 'Wednesday', 'Thursday'"

In [10]:
# Create an enumerated datatype for the weekday column
cur.execute("CREATE TYPE weekday_enum AS ENUM (" + weekdays + ");")

## 4. Creating a table

We'll create a table named `boston_crimes` inside the `crimes` schema of the `crime_db` database.

In [11]:
# Create the boston_crimes table
cur.execute("""
CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INT2,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekday_enum,
    lat DECIMAL,
    long DECIMAL
    );
    """
           )

## 5. Loading data into table

Now that we have created the table, we can load the data into it.

In [12]:
# Load data from boston.csv file into the boston_crimes table
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)

In [13]:
# Confirm addition of data
cur.execute("""
SELECT *
  FROM crimes.boston_crimes
 LIMIT 3;
 """)

print(cur.fetchone(), '\n')
print(cur.description)

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053')) 

(Column(name='incident_number', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='offense_code', type_code=21, display_size=None, internal_size=2, precision=None, scale=None, null_ok=None), Column(name='description', type_code=1043, display_size=None, internal_size=100, precision=None, scale=None, null_ok=None), Column(name='date', type_code=1082, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='day_of_the_week', type_code=16473, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='lat', type_code=1700, display_size=None, internal_size=-1, precision=65535, scale=65535, null_ok=None), Column(name='long', type_code=1700, display_size=None, internal_size=-1, precision=65535, scale=65535, null_ok=None))


We can see that datatype of each column via the type_code value. We'll check these type_code values for consitency.

In [14]:
# Verify assigned datatypes
cur.execute("""
SELECT oid, typname
  FROM pg_catalog.pg_type
 WHERE oid IN (23,21,1043,1082,16517,1700);
 """)

cur.fetchall()

[(21, 'int2'),
 (23, 'int4'),
 (1043, 'varchar'),
 (1082, 'date'),
 (1700, 'numeric')]

## 6. Creating user groups

Our database is starting to look good! We have created a database with a schema inside it for hold data about crimes. We selected the right datatypes for storing the data, created a table and loaded the CSV containing crimes about Boston.

Now it is time to handle users. Our goal is to create the two user groups that we have learned about: `readonly` and `readwrite`. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the `public` group and on the `public` schema.

Both these uses will also need the following:

* Connection privileges on the `crime_db` database — otherwise, they won't be able to do anything.
* Usage of the `crimes` schema — this is where all data will be stored. 

### 6.1 Eliminating inherited privileges from public group and schema

In [15]:
# Revoke all privileges on the default public schema
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

# Revoke all privileges on the default public group
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

### 6.2 Creating the readonly group

In [16]:
# Create readonly group
cur.execute("CREATE GROUP readonly;")

# Grant readonly privileges to readonly group
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

# Grant connection privielges on the crime_db database to the readonly group
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")

# Grant usage of the crimes schema to the readonly group
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")

### 6.3 Creating the readwrite group

In [17]:
# Create readwrite group
cur.execute("CREATE GROUP readwrite;")

# Grant readwrte privileges to readwrite group
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

# Grant connection privielges on the crime_db database to the readwrite group
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

# Grant usage of the crimes schema to the readwrite group
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

## 7. Creating users

We are nearly done with setting up our crime database. The only thing that we need to do is create users.

We'll wrap up the project by creating one user in each group. We'll need to create each user and then assign them to each group. 

In [18]:
# Create users and assign them to appropriate
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## 8. Testing configurations and database setup

It is a good practice to test that everything is configured as expected when you finish setting up the database. 

We can use SQL queries to check whether the objects have been created and that users and groups have the right privileges. We can query the `pg_roles` table to inspect privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

We'll write a few tests to check that privileges are set accordingly now.

In [31]:
# Get column names of table_privileges table
cur.execute("""
SELECT column_name
  FROM information_schema.columns
 WHERE table_name = 'table_privileges';
""")

print(cur.fetchall(), '\n')

# Check privileges for readony and readwrite groups
cur.execute("""
SELECT *
  FROM information_schema.table_privileges
 WHERE grantee IN ('readonly', 'readwrite');
""")

for row in cur.fetchall():
    print(row)

[('grantor',), ('grantee',), ('table_catalog',), ('table_schema',), ('table_name',), ('privilege_type',), ('is_grantable',), ('with_hierarchy',)] 

('dq', 'readonly', 'crime_db', 'crimes', 'boston_crimes', 'SELECT', 'NO', 'YES')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'INSERT', 'NO', 'NO')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'SELECT', 'NO', 'YES')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'UPDATE', 'NO', 'NO')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'DELETE', 'NO', 'NO')


In [32]:
# Get column names of pg_user table
cur.execute("""
SELECT column_name
  FROM information_schema.columns
 WHERE table_name = 'pg_user';
""")

print(cur.fetchall(), '\n')

# Check users privileges
cur.execute("""
SELECT *
  FROM pg_user;
""")

for row in cur.fetchall():
    print(row)

[('usename',), ('usesysid',), ('usecreatedb',), ('usesuper',), ('usecatupd',), ('userepl',), ('passwd',), ('valuntil',), ('useconfig',)] 

('dq', 10, True, True, True, True, '********', None, None)
('data_analyst', 16497, False, False, False, False, '********', None, None)
('data_scientist', 16498, False, False, False, False, '********', None, None)


## Conclusion

We used PostgreSQl to create a database, `crimes_db`, for storing data related to crimes commited in Boston, Massachusetts. To do so, we:

1. Created the schema, `crimes` and the table, `boston_crimes` within the `crimes_db` database.
2. Explored the data from `boston.csv` to determine appropriate datatypes
3. Created an enumerated datatype for the `day_of_the_week` column.
4. Added columns to the `boston_crimes` table with the appropriate datatypes and `PRIMARY KEY`.
5. Loaded the data from `boston.csv` into the `boston_crimes` table.
6. Created user groups `readonly` and `readwrite` while assigning the appropriate privieges.
7. Created users `data_analyst` and `data_scientist` and assigned them to the appropriate groups.
8. Checked that these new configurations met our expectations using PostgreSQL internal tables.

### Next steps

* Add more data, users, and groups with the appropriate privileges and datatypes.
* Set up a database locally